In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import eval_extraction
import openai
import extract
openai.api_key = open('/home/chansingh/.OPENAI_KEY').read().strip()

# Load the data

In [2]:
df = pd.read_pickle("../data/data_clean.pkl")[
    [
        "id",
        "paper___url",
        "paper___raw_text",
        "paper___year",
        "participants___male",
        "participants___female",
        "participants___total",
        "participants___white",
        "participants___black",
        "participants___latino",
        "participants___asian",
    ]
]

In [3]:
df.tail(n=300)

,id,paper___url,paper___raw_text,paper___year,participants___male,participants___female,participants___total,participants___white,participants___black,participants___latino,participants___asian
390,2076,https://www.ncbi.nlm.nih.gov/pubmed/12520701,"See discussions, stats, and author profiles fo...",2003,100,0,100,-1,-1,-1,-1
391,1985,http://www.ncbi.nlm.nih.gov/pubmed/12924748,Full Terms & Conditions of access and use can ...,2003,-1,-1,-1,-1,-1,-1,-1
392,780,https://www.ncbi.nlm.nih.gov/pubmed/17659153,46\nCJEM • JCMU\nJanuary • janvier 2003; 5 (1)...,2003,-1,-1,-1,-1,-1,-1,-1
393,324,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,RESPIRATORY INFECTION\nDefining community acqu...,2003,550,518,1068,-1,-1,-1,-1
394,4012,https://www.ncbi.nlm.nih.gov/pubmed/12765673,Clinical Nutrition (2003) 22(3): 321–336\nr 20...,2002,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...
685,10187,NaN,None,-1,-1,-1,-1,-1,-1,-1,-1
686,3401,NaN,None,-1,-1,-1,-1,-1,-1,-1,-1
687,2048,NaN,None,-1,-1,-1,-1,-1,-1,-1,-1
688,2040,NaN,None,-1,-1,-1,-1,-1,-1,-1,-1


# Make predictions

Current outputs:
- num_male, num_female, num_total, num_male_evidence_span, num_female_evidence_span, num_total_evidence_span
- num_white, num_black, num_latino, num_asian, race_evidence_span

Targets:
- 'participants___male', 'participants___female', 'participants___total'
- 'participants___white', 'participants___black', 'participants___latino', 'participants___asian'

In [5]:
idxs = df['paper___raw_text'].notna()
texts = df[idxs]['paper___raw_text'].tolist()
extractions = extract.extract_nums_df(texts)
for k in extractions.keys():
    df.loc[idxs, k] = extractions[k]

attempting to add dict_keys(['num_male', 'num_female', 'num_total', 'evidence_span_gender'])


0it [00:00, ?it/s]

542it [00:00, 1203.68it/s]


attempting to add dict_keys(['num_white', 'num_black', 'num_latino', 'num_asian', 'evidence_span_race'])


542it [00:00, 1325.68it/s]

Unterminated string starting at: line 6 column 23 (char 105)


### Evaluate
Evaluates whether each extracted number is within 1 of the human-labeled value

In [18]:
# Apply some postprocessing to the predictions (convert percentages)
gender_count_processed = df.apply(eval_extraction.process_gender_counts, axis=1)
df["num_male"] = gender_count_processed.apply(lambda x: x[0]).astype(str)
df["num_female"] = gender_count_processed.apply(lambda x: x[1]).astype(str)
# print("Total n (with paper text)", idxs.sum())
eval_extraction.compute_metrics_within_1(
    df,
    preds_col_to_gt_col_dict={
        "num_male": "participants___male",
        "num_female": "participants___female",
        "num_total": "participants___total",
    },
)

,target,n_gt,n_pred,n_correct,recall,precision
0,participants___male,327,254,197,0.60,0.78
1,participants___female,328,257,199,0.61,0.77
2,participants___total,385,389,286,0.74,0.74
